# Learning 5: Drone Altitude Comparison

**Duration**: 15-20 minutes  
**Level**: Intermediate  
**Prerequisites**: [Learning 4: GSD Basics](learning_04_gsd_basics.ipynb), [Quickstart 2: Drone Basic](quickstart_02_drone_basic.ipynb)

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. **Compare** all 8 drone presets and understand their use cases
2. **Understand** the altitude vs coverage tradeoff
3. **Analyze** motion blur effects at different speeds
4. **Validate** GSD calculations against real drone specs (DJI Phantom style)
5. **Select** the appropriate preset for your application

---

## Why Altitude Matters

Altitude is the primary factor in aerial imaging:

- **Higher altitude** = wider coverage, lower detail (larger GSD)
- **Lower altitude** = narrower coverage, higher detail (smaller GSD)

The relationship is linear:

$$\text{GSD} = \frac{H \times p}{f}$$

Where:
- H = altitude (meters)
- p = pixel pitch (meters)
- f = focal length (meters)

This notebook compares all 8 SPIDS drone presets to help you choose the right one.

## Setup

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch

# SPIDS imports
from prism.core import create_checkerboard_target
from prism.core.instruments import create_instrument
from prism.scenarios import get_scenario_preset, list_scenario_presets


# Plotting style
plt.rcParams["figure.figsize"] = (14, 8)
plt.rcParams["font.size"] = 11

print("Setup complete!")

---

# Section 1: All 8 Drone Presets Overview

SPIDS includes 8 drone camera presets covering a wide range of applications:
- Close-range inspection (10m)
- Detailed surveying (20m)
- Standard surveys (50m)
- Large-area mapping (100m+)

Let's load all presets and create a comprehensive comparison table.

In [ ]:
# Get all drone presets
drone_preset_names = list_scenario_presets(category="drone")

print(f"Found {len(drone_preset_names)} drone presets:")
for name in drone_preset_names:
    print(f"  - {name}")

In [ ]:
# Load all presets and extract parameters
presets_data = []

for name in drone_preset_names:
    scenario = get_scenario_preset(name)

    presets_data.append(
        {
            "name": name,
            "altitude_m": scenario.altitude_m,
            "lens": str(scenario._lens),
            "sensor": scenario._sensor.name,
            "gsd_cm": scenario.actual_gsd_cm,
            "swath_m": scenario.swath_width_m,
            "speed_mps": scenario.ground_speed_mps,
            "motion_blur_px": scenario.motion_blur_pixels,
            "fresnel_number": scenario.fresnel_number,
        }
    )

# Sort by altitude
presets_data.sort(key=lambda x: x["altitude_m"])

# Print comprehensive comparison table
print("=" * 120)
print("DRONE PRESET COMPARISON TABLE")
print("=" * 120)
print(
    f"{'Preset':<28} {'Alt (m)':<8} {'Lens':<14} {'Sensor':<14} {'GSD (cm)':<10} {'Swath (m)':<10} {'Speed':<8} {'Use Case'}"
)
print("-" * 120)

use_cases = {
    "drone_10m_inspection": "Close inspection",
    "drone_20m_detail": "Detailed surveying",
    "drone_infrastructure_30m": "Infrastructure",
    "drone_agriculture_50m": "Agriculture",
    "drone_50m_survey": "Site survey",
    "drone_hover_50m": "Precision hover",
    "drone_100m_mapping": "Large area mapping",
    "drone_phantom_120m": "Consumer drone",
}

for p in presets_data:
    use_case = use_cases.get(p["name"], "General")
    speed_str = f"{p['speed_mps']:.0f} m/s" if p["speed_mps"] > 0 else "Hover"
    print(
        f"  {p['name']:<26} {p['altitude_m']:<8.0f} {p['lens']:<14} {p['sensor']:<14} {p['gsd_cm']:<10.2f} {p['swath_m']:<10.1f} {speed_str:<8} {use_case}"
    )

print("=" * 120)

In [ ]:
# Visualize GSD vs Altitude for all presets
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

altitudes = [p["altitude_m"] for p in presets_data]
gsds = [p["gsd_cm"] for p in presets_data]
swaths = [p["swath_m"] for p in presets_data]
names = [p["name"].replace("drone_", "") for p in presets_data]

# GSD vs Altitude
ax1 = axes[0]
colors = plt.cm.viridis(np.linspace(0, 0.9, len(presets_data)))
bars1 = ax1.bar(range(len(presets_data)), gsds, color=colors, edgecolor="black", linewidth=0.5)

# Add altitude annotations
for i, (bar, alt) in enumerate(zip(bars1, altitudes)):
    ax1.annotate(
        f"{alt:.0f}m",
        (bar.get_x() + bar.get_width() / 2, bar.get_height()),
        ha="center",
        va="bottom",
        fontsize=9,
        fontweight="bold",
    )

ax1.set_xticks(range(len(presets_data)))
ax1.set_xticklabels(names, rotation=45, ha="right", fontsize=9)
ax1.set_ylabel("GSD (cm/pixel)", fontsize=12)
ax1.set_title("Ground Sampling Distance by Preset", fontsize=14, fontweight="bold")
ax1.axhline(5, color="green", linestyle="--", alpha=0.7, label="Inspection (<5cm)")
ax1.axhline(10, color="orange", linestyle="--", alpha=0.7, label="Survey (<10cm)")
ax1.legend(loc="upper left")
ax1.grid(axis="y", alpha=0.3)

# Swath Width vs Altitude
ax2 = axes[1]
bars2 = ax2.bar(range(len(presets_data)), swaths, color=colors, edgecolor="black", linewidth=0.5)

ax2.set_xticks(range(len(presets_data)))
ax2.set_xticklabels(names, rotation=45, ha="right", fontsize=9)
ax2.set_ylabel("Swath Width (m)", fontsize=12)
ax2.set_title("Coverage Width by Preset", fontsize=14, fontweight="bold")
ax2.grid(axis="y", alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey Observations:")
print("  - GSD increases linearly with altitude")
print("  - Swath width also increases with altitude")
print("  - Higher altitude = wider coverage but less detail")

---

# Section 2: Image Same Scene at Different Altitudes

The best way to understand the altitude tradeoff is to image the **same scene** from different altitudes.

We'll compare three presets:
- **10m inspection**: Ultra-fine detail (GSD ~1.1 cm)
- **50m survey**: Standard surveying (GSD ~6.5 cm)
- **100m mapping**: Large area coverage (GSD ~18.6 cm)

We'll use a checkerboard test pattern with features of known sizes.

In [ ]:
# Select three presets for comparison
comparison_presets = [
    "drone_10m_inspection",
    "drone_50m_survey",
    "drone_100m_mapping",
]

scenarios = {}
cameras = {}

print("Loading comparison presets...")
print("=" * 70)

for preset_name in comparison_presets:
    scenario = get_scenario_preset(preset_name)
    scenarios[preset_name] = scenario

    # Create camera instrument
    config = scenario.to_instrument_config()
    camera = create_instrument(config)
    cameras[preset_name] = camera

    print(f"\n{preset_name}:")
    print(f"  Altitude: {scenario.altitude_m:.0f} m")
    print(f"  GSD: {scenario.actual_gsd_cm:.2f} cm/pixel")
    print(f"  Swath: {scenario.swath_width_m:.1f} m")
    print(f"  Can resolve: {scenario.actual_gsd_cm * 2:.1f} cm features")

print("\n" + "=" * 70)

In [ ]:
# Create test scene - use the smallest swath to ensure all cameras can see it
# We'll use the 10m inspection preset's swath as reference
reference_scenario = scenarios["drone_10m_inspection"]
field_size = reference_scenario.swath_width_m  # ~6.7m

# Create checkerboard with 5cm squares - should be:
# - Well resolved at 10m (GSD 1.1cm -> ~4.5 pixels per square)
# - Marginally resolved at 50m (GSD 6.5cm -> ~0.8 pixels per square)
# - Unresolved at 100m (GSD 18.6cm -> ~0.3 pixels per square)
square_size = 0.05  # 5 cm squares
resolution = 1024

target = create_checkerboard_target(
    field_size=field_size,
    square_size=square_size,
    resolution=resolution,
    margin_ratio=0.25,
)

scene = target.generate()

print("Test Scene Properties:")
print(f"  Field of view: {field_size:.1f} m x {field_size:.1f} m")
print(f"  Square size: {square_size * 100:.0f} cm")
print(f"  Number of squares: {target.checker_config.n_squares}")
print("\nResolution expectations:")
print(
    f"  At 10m (GSD {scenarios['drone_10m_inspection'].actual_gsd_cm:.1f}cm): RESOLVED (squares >> GSD)"
)
print(
    f"  At 50m (GSD {scenarios['drone_50m_survey'].actual_gsd_cm:.1f}cm): MARGINAL (squares ~ GSD)"
)
print(
    f"  At 100m (GSD {scenarios['drone_100m_mapping'].actual_gsd_cm:.1f}cm): UNRESOLVED (squares < GSD)"
)

In [ ]:
# Image the scene with all three cameras
measurements = {}

input_field = scene.unsqueeze(0).unsqueeze(0)

for preset_name, camera in cameras.items():
    with torch.no_grad():
        measurement = camera.forward(input_field)
    measurements[preset_name] = measurement.squeeze().cpu().numpy()

print("All measurements complete!")

In [ ]:
# Three-way comparison visualization
fig, axes = plt.subplots(1, 4, figsize=(20, 5))
extent = [0, field_size, 0, field_size]

# Ground truth
axes[0].imshow(scene.cpu().numpy(), cmap="gray", extent=extent, origin="lower")
axes[0].set_title("Ground Truth\n(5cm squares)", fontsize=12, fontweight="bold")
axes[0].set_xlabel("Distance (m)")
axes[0].set_ylabel("Distance (m)")

# Three altitude comparisons
titles = [
    f"10m Altitude\nGSD: {scenarios['drone_10m_inspection'].actual_gsd_cm:.1f} cm",
    f"50m Altitude\nGSD: {scenarios['drone_50m_survey'].actual_gsd_cm:.1f} cm",
    f"100m Altitude\nGSD: {scenarios['drone_100m_mapping'].actual_gsd_cm:.1f} cm",
]

for idx, preset_name in enumerate(comparison_presets):
    ax = axes[idx + 1]
    ax.imshow(measurements[preset_name], cmap="gray", extent=extent, origin="lower")
    ax.set_title(titles[idx], fontsize=12, fontweight="bold")
    ax.set_xlabel("Distance (m)")
    if idx == 0:
        ax.set_ylabel("Distance (m)")

plt.tight_layout()
plt.show()

print("\nObservations:")
print("  10m altitude: 5cm squares are clearly resolved (~4.5 pixels per square)")
print("  50m altitude: 5cm squares are barely visible (squares ~ GSD)")
print("  100m altitude: 5cm squares are completely unresolved (squares < GSD)")
print("\nThis demonstrates the fundamental altitude vs detail tradeoff!")

In [ ]:
# Zoomed comparison - show what each altitude can resolve
fig, axes = plt.subplots(1, 4, figsize=(20, 5))

# Zoom to center region (1/4 of field)
center = resolution // 2
zoom_size = resolution // 4
zoom_slice = slice(center - zoom_size // 2, center + zoom_size // 2)
zoom_extent = field_size / 4

# Ground truth zoomed
axes[0].imshow(
    scene[zoom_slice, zoom_slice].cpu().numpy(),
    cmap="gray",
    extent=[0, zoom_extent, 0, zoom_extent],
    origin="lower",
)
axes[0].set_title("Ground Truth (Zoomed)\n5cm squares", fontsize=12, fontweight="bold")
axes[0].set_xlabel("Distance (m)")
axes[0].set_ylabel("Distance (m)")

# Annotations for what can be resolved
can_resolve = [
    "Can resolve: small cracks",
    "Can resolve: road markings",
    "Can resolve: vehicles",
]

for idx, preset_name in enumerate(comparison_presets):
    ax = axes[idx + 1]
    ax.imshow(
        measurements[preset_name][zoom_slice, zoom_slice],
        cmap="gray",
        extent=[0, zoom_extent, 0, zoom_extent],
        origin="lower",
    )
    ax.set_title(f"{titles[idx]}\n{can_resolve[idx]}", fontsize=11, fontweight="bold")
    ax.set_xlabel("Distance (m)")

plt.tight_layout()
plt.show()

print("\nFeature Detection Guidelines (approximate):")
print("-" * 60)
print(f"{'Altitude':<12} {'GSD':<10} {'Can Detect':<40}")
print("-" * 60)
print(f"{'10m':<12} {'~1 cm':<10} {'Small cracks, wires, detailed textures'}")
print(f"{'50m':<12} {'~6.5 cm':<10} {'Road markings, sidewalks, vegetation'}")
print(f"{'100m':<12} {'~19 cm':<10} {'Vehicles, buildings, large features'}")
print("-" * 60)

---

# Section 3: Motion Blur Analysis

When a drone is moving during image capture, **motion blur** degrades image quality.

Motion blur formula:
$$\text{blur (pixels)} = \frac{\text{speed} \times \text{exposure}}{\text{GSD}}$$

Key insight:
- Higher speed → more blur
- Longer exposure → more blur
- Larger GSD → less blur (in pixel terms)

SPIDS assumes a typical exposure of 1/1000s for sunny conditions.

In [ ]:
# Motion blur comparison
print("=" * 80)
print("MOTION BLUR ANALYSIS")
print("=" * 80)
print(f"{'Preset':<28} {'Speed (m/s)':<12} {'GSD (cm)':<10} {'Blur (px)':<10} {'Status'}")
print("-" * 80)

for p in presets_data:
    blur = p["motion_blur_px"]
    if blur == 0:
        status = "PERFECT (Hover)"
    elif blur < 0.5:
        status = "EXCELLENT"
    elif blur < 1.0:
        status = "GOOD"
    elif blur < 2.0:
        status = "ACCEPTABLE"
    else:
        status = "WARNING"

    speed_str = f"{p['speed_mps']:.0f}" if p["speed_mps"] > 0 else "0 (hover)"
    print(f"  {p['name']:<26} {speed_str:<12} {p['gsd_cm']:<10.2f} {blur:<10.2f} {status}")

print("=" * 80)
print("\nMotion Blur Guidelines:")
print("  < 0.5 pixels: Excellent - no visible degradation")
print("  0.5-1.0 pixels: Good - minimal impact")
print("  1.0-2.0 pixels: Acceptable - visible but usable")
print("  > 2.0 pixels: Warning - significant quality loss")

In [ ]:
# Compare hover vs moving drone at 50m
hover_scenario = get_scenario_preset("drone_hover_50m")
survey_scenario = get_scenario_preset("drone_50m_survey")

print("Hover Mode vs Survey Mode Comparison (50m altitude)")
print("=" * 60)
print(f"{'Parameter':<25} {'Hover Mode':<18} {'Survey Mode'}")
print("-" * 60)
print(
    f"{'Ground Speed':<25} {hover_scenario.ground_speed_mps} m/s{'':<14} {survey_scenario.ground_speed_mps} m/s"
)
print(
    f"{'GSD':<25} {hover_scenario.actual_gsd_cm:.2f} cm{'':<13} {survey_scenario.actual_gsd_cm:.2f} cm"
)
print(
    f"{'Motion Blur':<25} {hover_scenario.motion_blur_pixels:.2f} pixels{'':<10} {survey_scenario.motion_blur_pixels:.2f} pixels"
)
print(f"{'Best For':<25} {'Precision imaging':<18} {'Area coverage'}")
print("=" * 60)
print("\nUse hover mode when:")
print("  - Maximum sharpness is required")
print("  - Inspecting critical infrastructure")
print("  - Time is not a constraint")
print("\nUse survey mode when:")
print("  - Covering large areas efficiently")
print("  - Motion blur is acceptable")
print("  - Time/battery constraints exist")

In [ ]:
# Visualize motion blur levels
fig, ax = plt.subplots(figsize=(14, 6))

# Extract data
names = [p["name"].replace("drone_", "") for p in presets_data]
speeds = [p["speed_mps"] for p in presets_data]
blurs = [p["motion_blur_px"] for p in presets_data]

# Color by blur level
colors = []
for blur in blurs:
    if blur == 0:
        colors.append("green")
    elif blur < 0.5:
        colors.append("limegreen")
    elif blur < 1.0:
        colors.append("yellow")
    elif blur < 2.0:
        colors.append("orange")
    else:
        colors.append("red")

bars = ax.bar(range(len(presets_data)), blurs, color=colors, edgecolor="black", linewidth=0.5)

# Add speed annotations
for i, (bar, speed) in enumerate(zip(bars, speeds)):
    speed_str = f"{speed:.0f} m/s" if speed > 0 else "Hover"
    ax.annotate(
        speed_str,
        (bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.02),
        ha="center",
        va="bottom",
        fontsize=9,
    )

# Threshold lines
ax.axhline(0.5, color="green", linestyle="--", alpha=0.5, label="Excellent (<0.5)")
ax.axhline(1.0, color="orange", linestyle="--", alpha=0.5, label="Acceptable (<1.0)")
ax.axhline(2.0, color="red", linestyle="--", alpha=0.5, label="Warning (>2.0)")

ax.set_xticks(range(len(presets_data)))
ax.set_xticklabels(names, rotation=45, ha="right", fontsize=9)
ax.set_ylabel("Motion Blur (pixels)", fontsize=12)
ax.set_title("Motion Blur by Preset (exposure = 1/1000s)", fontsize=14, fontweight="bold")
ax.legend(loc="upper right")
ax.grid(axis="y", alpha=0.3)

plt.tight_layout()
plt.show()

---

# Section 4: DJI Phantom Validation

Let's validate SPIDS against real-world drone specs.

The `drone_phantom_120m` preset is designed to match DJI Phantom 4 Pro characteristics:
- **Sensor**: 1" CMOS (13.2 × 8.8 mm)
- **Lens**: 24mm equivalent (8.8mm actual), f/2.8
- **Resolution**: 20 MP
- **Max altitude**: ~120m (legal limit in many regions)

In [ ]:
# Load DJI-style preset
phantom = get_scenario_preset("drone_phantom_120m")

print("DJI Phantom 4 Pro Style Validation")
print("=" * 70)
print("\nSPIDS Preset: drone_phantom_120m")
print("-" * 70)

# SPIDS values
print(f"\n{'Parameter':<30} {'SPIDS Value':<20} {'DJI Spec (approx)'}")
print("-" * 70)
print(f"{'Sensor Type':<30} {phantom._sensor.name:<20} {'1-inch CMOS'}")
print(
    f"{'Sensor Size':<30} {phantom._sensor.width_mm:.1f} x {phantom._sensor.height_mm:.1f} mm{'':<5} {'13.2 x 8.8 mm'}"
)
print(f"{'Pixel Pitch':<30} {phantom._sensor.pixel_pitch_um:.1f} um{'':<14} {'~2.4 um'}")
print(f"{'Focal Length':<30} {phantom._lens.focal_length_mm:.0f} mm{'':<15} {'24mm equiv (8.8mm)'}")
print(f"{'F-number':<30} f/{phantom._lens.f_number}{'':<16} {'f/2.8'}")
print(f"{'Altitude':<30} {phantom.altitude_m:.0f} m{'':<16} {'120m (legal max)'}")
print(f"{'GSD at altitude':<30} {phantom.actual_gsd_cm:.2f} cm{'':<14} {'~3-5 cm typical'}")
print("-" * 70)

# Calculate expected GSD for verification
# GSD = H * p / f
expected_gsd_cm = (
    (phantom.altitude_m * phantom._sensor.pixel_pitch_um * 1e-6)
    / (phantom._lens.focal_length_mm * 1e-3)
    * 100
)
print("\nGSD Verification:")
print(
    f"  Calculated: GSD = ({phantom.altitude_m}m * {phantom._sensor.pixel_pitch_um}um) / {phantom._lens.focal_length_mm}mm"
)
print(f"           = {expected_gsd_cm:.2f} cm/pixel")
print(f"  SPIDS reports: {phantom.actual_gsd_cm:.2f} cm/pixel")
print(f"  Match: {'YES' if abs(expected_gsd_cm - phantom.actual_gsd_cm) < 0.01 else 'NO'}")

---

# Assessment Questions

Test your understanding:

In [ ]:
print("=" * 70)
print("ASSESSMENT QUESTIONS")
print("=" * 70)

print("\nQuestion 1: Which preset for 10cm detail at 100m?")
print("-" * 50)
print("  Answer: drone_100m_mapping")
print(f"  Reason: GSD = {scenarios['drone_100m_mapping'].actual_gsd_cm:.1f} cm at 100m altitude")
print("  To resolve 10cm features, need GSD < 5cm (Nyquist)")
print("  This preset has GSD ~19cm, so 10cm features are MARGINAL")
print("  For reliable 10cm detection, use drone_50m_survey at 50m instead")

print("\n" + "-" * 70)

print("\nQuestion 2: Why use hover mode?")
print("-" * 50)
print("  Answer: To eliminate motion blur")
print(f"  - Hover mode: {hover_scenario.motion_blur_pixels:.2f} pixels blur")
print(f"  - Survey mode: {survey_scenario.motion_blur_pixels:.2f} pixels blur")
print("  - Use when maximum sharpness is critical")
print("  - Trade-off: slower coverage, more battery usage")

print("\n" + "-" * 70)

print("\nQuestion 3: Trade-off: high altitude vs low altitude?")
print("-" * 50)
print("  High altitude (100m+):")
print("    + Wide swath width (>100m coverage)")
print("    + Faster area mapping")
print("    + Fewer flight lines needed")
print("    - Lower detail (GSD >15cm)")
print("    - Cannot detect small features")
print("  ")
print("  Low altitude (10-30m):")
print("    + High detail (GSD <5cm)")
print("    + Can detect small defects")
print("    - Narrow swath (<20m)")
print("    - More flight lines needed")
print("    - Longer mission time")

print("\n" + "=" * 70)

---

# Summary

## Key Takeaways

### 1. Preset Selection Guide

| Application | Recommended Preset | GSD | Altitude |
|------------|-------------------|-----|----------|
| Close inspection | `drone_10m_inspection` | ~1 cm | 10m |
| Detailed survey | `drone_20m_detail` | ~1.6 cm | 20m |
| Infrastructure | `drone_infrastructure_30m` | ~4 cm | 30m |
| Agriculture | `drone_agriculture_50m` | ~5.6 cm | 50m |
| Site survey | `drone_50m_survey` | ~6.5 cm | 50m |
| Precision work | `drone_hover_50m` | ~6.5 cm | 50m (no blur) |
| Large mapping | `drone_100m_mapping` | ~18.6 cm | 100m |
| Consumer drone | `drone_phantom_120m` | ~8 cm | 120m |

### 2. Altitude vs Coverage Tradeoff

- **GSD = H × p / f** (linear relationship with altitude)
- Double the altitude → double the GSD
- Higher altitude = wider coverage, less detail

### 3. Motion Blur Guidelines

- **blur (px) = speed × exposure / GSD**
- < 0.5 px: Excellent (no visible effect)
- 0.5-1.0 px: Good (minimal impact)
- 1.0-2.0 px: Acceptable (some degradation)
- > 2.0 px: Warning (significant blur)

### 4. Feature Detection Rule of Thumb

To reliably detect a feature, its size should be **at least 2× the GSD** (Nyquist criterion).

---

## Reference Documentation

For detailed parameter lookup:

- **[Scenario Preset Catalog](../../docs/references/scenario_preset_catalog.md)** - All 17 presets with specifications
- **[Drone Camera Parameters](../../docs/references/drone_camera_parameters.md)** - Sensor specs, lens options
- **[Fresnel Propagation Regimes](../../docs/references/fresnel_propagation_regimes.md)** - When diffraction matters

---

## Next Steps

- **[Learning 4: GSD Basics](learning_04_gsd_basics.ipynb)**: Deep dive into GSD calculations
- **[Quickstart 2: Drone Basic](quickstart_02_drone_basic.ipynb)**: Hands-on drone imaging
- **[07_drone_mapping.py](../python_api/07_drone_mapping.py)**: Production drone workflow

In [ ]:
print("\n" + "=" * 70)
print("Congratulations! You've completed Learning 5: Drone Altitude Comparison")
print("=" * 70)
print("\nYou now understand:")
print("  [ok] All 8 drone presets and their use cases")
print("  [ok] Altitude vs coverage tradeoff")
print("  [ok] Motion blur effects and hover mode benefits")
print("  [ok] How to select the right preset for your application")
print("  [ok] GSD validation against real drone specs")
print("\nReady for drone mapping? Check out the production script!")